# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,39.02,91,99,3.02,GS,1685673414
1,1,aripuana,-9.1667,-60.6333,71.92,99,100,2.06,BR,1685673414
2,2,morris,40.8334,-74.5329,68.85,74,0,0.00,US,1685673414
3,3,jamestown,42.0970,-79.2353,74.84,46,0,8.05,US,1685673238
4,4,malindi,-3.2175,40.1191,76.06,86,6,8.59,KE,1685673209


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
p1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "EsriUSATopo",
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    color = 'City',
    alpha = 0.7
)

# Display the map
p1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
clean_df = city_data_df.loc[city_data_df['Max Temp'] <= 40]

# Drop any rows with null values

clean_df = clean_df.dropna()

# Display sample data
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,39.02,91,99,3.02,GS,1685673414
21,21,puerto natales,-51.7236,-72.4875,35.01,87,100,3.18,CL,1685673419
24,24,port-aux-francais,-49.3500,70.2167,38.53,77,100,27.02,TF,1685673420
66,66,kuusamo,65.9667,29.1833,33.84,93,100,11.50,FI,1685673440
92,92,ilulissat,69.2167,-51.1000,32.02,93,100,10.36,GL,1685673448
118,118,iqaluit,63.7506,-68.5145,28.13,80,100,13.80,CA,1685673266
119,119,happy valley-goose bay,53.3168,-60.3315,39.51,65,100,13.80,CA,1685673243
159,159,el calafate,-50.3408,-72.2768,37.40,82,100,3.89,AR,1685673469
160,160,longyearbyen,78.2186,15.6401,31.84,64,0,9.22,SJ,1685673470
161,161,peterhead,57.5058,-1.7981,37.80,79,1,5.57,GB,1685673231


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

C:\Users\alice\AppData\Local\Temp\ipykernel_9484\1385115104.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,91,
21,puerto natales,CL,-51.7236,-72.4875,87,
24,port-aux-francais,TF,-49.3500,70.2167,77,
66,kuusamo,FI,65.9667,29.1833,93,
92,ilulissat,GL,69.2167,-51.1000,93,
118,iqaluit,CA,63.7506,-68.5145,80,
119,happy valley-goose bay,CA,53.3168,-60.3315,65,
159,el calafate,AR,-50.3408,-72.2768,82,
160,longyearbyen,SJ,78.2186,15.6401,64,
161,peterhead,GB,57.5058,-1.7981,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000 #10,000 meters
params = {
    "categories": "accommodation.hotel",
    'apiKey': geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, 'Lat']
    lon = hotel_df.loc[index, 'Lng']

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
port-aux-francais - nearest hotel: Keravel
kuusamo - nearest hotel: Holiday Club Kuusamon Tropiikki
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
iqaluit - nearest hotel: Frobisher Inn
happy valley-goose bay - nearest hotel: Hotel North Two
el calafate - nearest hotel: Hostería Las Avutardas
longyearbyen - nearest hotel: The Vault
peterhead - nearest hotel: Albert Hotel
zelenoborskiy - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
polyarnyy - nearest hotel: No hotel found
nuuk - nearest hotel: Nordbo
hammerfest - nearest hotel: Thon Hotel Hammerfest
aasiaat - nearest hotel: SØMA
harstad - nearest hotel: Scandic Harstad
kostomuksha - nearest hotel: Хозяюшка
gadzhiyevo - nearest hotel: Оазис
panevezys - nearest hotel: Kranto Luxe Stay
quthing - nearest hotel: Mountainside Hotel
sisimiut - n

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,91,No hotel found
21,puerto natales,CL,-51.7236,-72.4875,87,Dorotea Patagonia Hostel
24,port-aux-francais,TF,-49.3500,70.2167,77,Keravel
66,kuusamo,FI,65.9667,29.1833,93,Holiday Club Kuusamon Tropiikki
92,ilulissat,GL,69.2167,-51.1000,93,Best Western Plus Hotel Ilulissat
118,iqaluit,CA,63.7506,-68.5145,80,Frobisher Inn
119,happy valley-goose bay,CA,53.3168,-60.3315,65,Hotel North Two
159,el calafate,AR,-50.3408,-72.2768,82,Hostería Las Avutardas
160,longyearbyen,SJ,78.2186,15.6401,64,The Vault
161,peterhead,GB,57.5058,-1.7981,79,Albert Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
p2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    alpha = 0.7,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
p2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)